In [1]:
a =1
b = 2
print("Soma:", a + b)

Soma: 3


In [4]:
print("Olá")

Olá


In [1]:
import findspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg
from pyspark.sql.types import *

# Inicializando a SparkSession
spark = SparkSession.builder \
    .appName("Exemplo PySpark") \
    .master("spark://vscode-master:7077") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/20 21:34:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

# Criando um DataFrame a partir de uma lista de dados
dados = [
    (1, "Alice", 25, "São Paulo"),
    (2, "Bob", 30, "Rio de Janeiro"),
    (3, "Carlos", 35, "São Paulo"),
    (4, "Daniela", 28, "Belo Horizonte"),
    (5, "Eduardo", 40, "Rio de Janeiro")
]

# Definindo o schema do DataFrame
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("nome", StringType(), False),
    StructField("idade", IntegerType(), False),
    StructField("cidade", StringType(), False)
])

# Criando o DataFrame
df = spark.createDataFrame(dados, schema)

# Exibindo o schema e os dados
print("Schema do DataFrame:")
df.printSchema()

print("Dados do DataFrame:")
df.show()

# Realizando transformações
# Filtro: Pessoas com mais de 30 anos
df_filtrado = df.filter(col("idade") > 30)

# Agregação: Média de idade por cidade
df_agregado = df.groupBy("cidade").agg(avg("idade").alias("media_idade"))

# Exibindo os resultados
print("Dados filtrados (idade > 30):")
df_filtrado.show()

print("Média de idade por cidade:")
df_agregado.show()

# Salvando o resultado em um arquivo Parquet
df_agregado.write.parquet("resultado_agregado.parquet")
print("Resultado salvo em 'resultado_agregado.parquet'")


Schema do DataFrame:
root
 |-- id: integer (nullable = false)
 |-- nome: string (nullable = false)
 |-- idade: integer (nullable = false)
 |-- cidade: string (nullable = false)

Dados do DataFrame:


+---+-------+-----+--------------+
| id|   nome|idade|        cidade|
+---+-------+-----+--------------+
|  1|  Alice|   25|     São Paulo|
|  2|    Bob|   30|Rio de Janeiro|
|  3| Carlos|   35|     São Paulo|
|  4|Daniela|   28|Belo Horizonte|
|  5|Eduardo|   40|Rio de Janeiro|
+---+-------+-----+--------------+

Dados filtrados (idade > 30):
+---+-------+-----+--------------+
| id|   nome|idade|        cidade|
+---+-------+-----+--------------+
|  3| Carlos|   35|     São Paulo|
|  5|Eduardo|   40|Rio de Janeiro|
+---+-------+-----+--------------+

Média de idade por cidade:


+--------------+-----------+
|        cidade|media_idade|
+--------------+-----------+
|     São Paulo|       30.0|
|Rio de Janeiro|       35.0|
|Belo Horizonte|       28.0|
+--------------+-----------+



Resultado salvo em 'resultado_agregado.parquet'


In [ ]:

# Encerrando a SparkSession
spark.stop()